In [2]:
from music21 import stream, interval, corpus, instrument
from music21 import converter, note, chord, environment, duration
import notebook
import argparse
import pandas as pd
import pathlib
import numpy as np
import seaborn as sns
import matplotlib as plt
import turtle as ttl
import random
%matplotlib inline
verbose = 0

## Chaos Game references
### Wikipedia
[Iterated Function Systems](https://en.wikipedia.org/wiki/Iterated_function_system)

[Chaos Game](https://en.wikipedia.org/wiki/Chaos_game)

[Barnsley Fern](https://en.wikipedia.org/wiki/Barnsley_fern)

[L-System](https://en.wikipedia.org/wiki/L-system)

[Turtle graphics](https://en.wikipedia.org/wiki/Turtle_graphics)


### Python Packages

[turtle graphics](https://docs.python.org/3/library/turtle.html)<br>
Python 3.10 lib/turtle.py documentation

[The Chaos Game](https://beltoforion.de/en/recreational_mathematics/chaos_game.php)<br>
An implementation of the Chaos Game using polygons

[L-Systems Turtle Graphics Renderer](http://www.kevs3d.co.uk/dev/lsystems/#)

In [3]:
import turtle as ttl
import random
draw = True
verbose = 0

#
# An example IFS (Barnsley Fern) drawn using turtle graphics
#
if draw:
    pen = ttl.Turtle()
    ttl.clearscreen()
    pen.speed(0)
    pen.color("green")
    pen.penup()

x = 0
y = 0
limit = 500   # set to 11000 for more detail
for n in range(limit):
    sx = 65 * x
    sy = 37 * y -252  # scale the fern to fit nicely inside the window
    if draw:
        pen.goto(sx, sy)
        pen.pendown()
        pen.dot(3)
        pen.penup()
    r = random.random()
    if verbose > 0:
        print('r={} \tx,y = {},{}, \tscaled = {},{}'.format(r,x,y,sx,sy))

    if r < 0.01:
        x, y =  0.00 * x + 0.00 * y,  0.00 * x + 0.16 * y + 0.00
    elif r < 0.86:
        x, y =  0.85 * x + 0.04 * y, -0.04 * x + 0.85 * y + 1.60
    elif r < 0.93:
        x, y =  0.20 * x - 0.26 * y,  0.23 * x + 0.22 * y + 1.60
    else:
        x, y = -0.15 * x + 0.28 * y,  0.26 * x + 0.24 * y + 0.44
        
print("Done")
ttl.exitonclick()
ttl.done()


Done


Terminator: 

In [3]:
import pandas as pd

class LinearFunction(object):
    """Defines a linear function f(x,y) =  (ax + by + c, dx + ey + f)
        The function name is 'f1', 'f2', etc. and needs to be unique
        as a member of an iterated function system (IFS)
    """
    columns = ['a', 'b', 'c', 'd', 'e', 'f', 'p']
    round_places = 4
    
    def __init__(self, x_coeficients:(float), y_coeficients:(float), probability = 0.0):
        self.name = None
        self.probability = probability
        self.x_coeficients = x_coeficients   # a 3-tupple: a,b,c
        self.y_coeficients = y_coeficients   # a 3-tupple: d,e,f
        coefs = list(x_coeficients + y_coeficients)    # all the coeficients + the probability as a Series
        coefs.append(probability)
        self.coeficients = pd.Series(coefs)
        self.coeficients.index = LinearFunction.columns
    
    def __str__(self):
        if self.name is None:
            return f"f=({self.x_coeficients}), ({self.y_coeficients})"
        else:
            return f"{self.name}=({self.x_coeficients}), ({self.y_coeficients})"
    
    def evaluate_at(self, point):
        x = point[0]
        y = point[1]
        nx = self.x_coeficients[0]*x + self.x_coeficients[1]*y + self.x_coeficients[2]
        ny = self.y_coeficients[0]*x + self.y_coeficients[1]*y + self.y_coeficients[2]
        return (round(nx, LinearFunction.round_places), round(ny, LinearFunction.round_places))
        
        
class IFS(object):
    """Iterated Function System
    
        An IFS is a system of linear functions (techincally an affine transformation) 
        implemented as a DataFrame where each row defines a linear function:
        f(x,y) = (ax + by + c, dx + ey + d)
        Associated with each function is a probability p, such that the sum of
        all probabilities is 1. In the chaos game, this is the probability of
        that function being selected.
        
    """
    
    def __init__(self, lf=None, probability:float = 0.0):
        self.ifs_df = pd.DataFrame(columns=LinearFunction.columns)
        self.functions = []   # list of LinearFunction
        if lf is not None:
            self.add_function(lf, probability)
        
    def add_function(self, linear_function:LinearFunction, probability:float):
        num = len(self.ifs_df)
        linear_function.name = f"f{num}"
        self.functions.append(linear_function)
        coefs = linear_function.coeficients
        coefs['p'] = probability
        coefs['linear_function'] = linear_function
        coefs['name'] = linear_function.name
        self.ifs_df = self.ifs_df.append(coefs, ignore_index=True)
        
    def size(self):
        return len(self.ifs_df)
    
    def index(self, ind):
        return self.ifs_df.iloc[ind]['linear_function']
        

In [4]:
class PointScaler(object):
    """ Scales a (x,y) point
        Arguments:
            gridsize - the overall size as an integer ordered pair. Default is (600, 600)
            point_stats is a dict with keys 'minX', 'minY', 'maxX', 'maxY'
        The origin is the center of the grid, so the range of x-values is (-gridsize[0]/2, gridsize[0]/2)
        and y-values (-gridsize[1]/2, gridsize[1]/2)
    """
    
    def __init__(self, point_stats, gridsize = (600, 600)):
        self.gridsize = gridsize
        self.set_grid_size(gridsize, point_stats=point_stats)
    
    def set_grid_size(self, gridsize:(int,int), point_stats = None):
        self.xrange = (-gridsize[0]/2, gridsize[0]/2)
        self.yrange = (-gridsize[1]/2, gridsize[1]/2)
        #
        # if point_stats is None, use existing value
        #
        if point_stats is not None:
            self.point_stats = point_stats
        self.xspan = gridsize[0]
        self.yspan = gridsize[1]
        self.xPointSpan = point_stats['maxX'] - point_stats['minX']
        self.yPointSpan = point_stats['maxY'] - point_stats['minY']
    
    def scale(self, point) -> (int,int):
        x = point[0]
        y = point[1]
        sx = ((x - self.point_stats['minX']) / self.xPointSpan * self.xspan ) + self.xrange[0]
        sy = ((y - self.point_stats['minY']) / self.yPointSpan * self.yspan ) + self.yrange[0]
        return round(sx),round(sy)

In [5]:
import turtle as ttl
import random

class ChaosGame(object):
    """ Create and run a Chaos Game.
    Pseudo code:
        (x, y), = a random point in the biunit square
        iterate {
            i = a random integer from 0 to n to 1 inclusive (randomly select the function)
            (x, y) = Fi(x,y)
            plot x y( ), except during the first 20 iterations
        }
    
    """
    
    def __init__(self, ifs:IFS, size={'rows':600, 'columns':600}, draw=False, verbose = 0, point_scaler = None, reps = 1):
        self.verbose = verbose
        self.ifs = ifs
        self.size = size
        self.pen = None
        self.prob_sums = ifs.ifs_df['p'].cumsum()
        self.draw = draw
        self.reps = reps   # number of times to repeat the game
        # TODO replace with a color gradient
        self.colors = ['red', 'blue', 'yellow', 'darkred', 'green', 'orange', 'purple', 'pink', 'darkgreen',  'navy']
        self.points_df = pd.DataFrame(columns=['point', 'x', 'y', 'scaled_point', 'scaleX', 'scaleY', 'count'])
        self.min_max_dict = None
        self.point_scaler = point_scaler    # a PointScaler instance
        self.point_histogram = None
    
    def scale_point(self, point):
        """Scale a point using the configured point_scaler instance.
            If there it's None, return the point as is (unscaled)
        """
        if self.point_scaler is None:
            sx = point[0]
            sy = point[1]
            return (sx, sy)
        else:
            return self.point_scaler.scale(point)
    
    def select_function(self):
            #
            # select a linear function
            #
            r = random.random()
            x =self.prob_sums >= r
            lf_to_select = x.tolist().index(True)    # the index of the first True in the list
            linear_function = self.ifs.index(lf_to_select)
            return linear_function
    
    def point_range(self):
        mm = self.min_max()
        return round(abs(mm['maxX']-mm['minX']), 4), round(abs(mm['maxY']-mm['minY']),4)
        
    def min_max(self):
        if self.min_max_dict is None:
            self.min_max_dict = {'minX': self.points_df['x'].min(), 'minY':self.points_df['y'].min(),\
            'maxX': self.points_df['x'].max(), 'maxY':self.points_df['y'].max()}
        return self.min_max_dict
    
    def record_point(self, point, scaled_point) -> int:
        """Saves the point and scaled coordinates to points_df.
            If there is a point_scaler and the scaled_point already exists, 
            it increments the corresponding count otherwise it adds a new row with a count of 1.
        """
        count = 0
        self.scaled_point = scaled_point  # save the latest value
        if len(self.points_df) > 0 and self.point_scaler is not None:
            bs = self.points_df['scaled_point'].isin([scaled_point])
            ind = bs.idxmax()
            if bs.iloc[ind]:
                count = self.points_df.iloc[ind]['count'] + 1
                self.points_df.loc[(ind, 'count')]= count
        if count == 0:
            row = pd.Series(data={'point':point, 'x':point[0], 'y':point[1], 'scaled_point':scaled_point,\
                  'scaleX':scaled_point[0], 'scaleY':scaled_point[1], 'count':1})
            self.points_df = self.points_df.append(row, ignore_index=True)
        return count
    
    def histogram(self):
        """Creates a DataFrame from scaleX, scaleY and count columns of points_df
            Order is essentially bottom to top, left to right.
        """
        if self.point_histogram is None:
            self.point_histogram = self.points_df[['scaleX','scaleY','count']].sort_values(by=['scaleX','scaleY'], ignore_index=True)
        return self.point_histogram
    
    def draw_points(self):
        """Draws the existing scaled points
        
        """
        if len(self.points_df) > 0 and self.point_scaler is not None:
            self.pen = ttl.Turtle()
            # ttl.clearscreen()
            self.pen.speed(0)    # 0=the fastest, 10=fast, 6=normal, 3=slow, 1 = slowest
            self.pen.penup()
            for row in self.histogram().itertuples():
                sx = row.scaleX
                sy = row.scaleY
                count = row.count
                color_index = count-1 % len(self.colors)
                self.pen.color(self.colors[color_index])
                self.pen.goto(sx, sy)
                self.pen.pendown()
                self.pen.dot(3)
                self.pen.penup()
            print("Done")
            ttl.exitonclick()
            ttl.done()
                        
    def run(self, limit=500):
        if self.draw:    # draw while you compute
            self.pen = ttl.Turtle()
            ttl.clearscreen()
            self.pen.speed(0)    # 0=the fastest, 10=fast, 6=normal, 3=slow, 1 = slowest
            self.pen.penup()
        limit = limit + 21   # because we skip the first 20 iterations
        for i in range(self.reps):
            if self.draw:
                self.pen.color(self.colors[i])
            point =  (random.uniform(-1,1),random.uniform(-1,1))  # pick a point in biunit square [-1, 1]
            count = 0
            for n in range(limit):
                #
                # select a linear function
                # 
                linear_function = self.select_function()
                point = linear_function.evaluate_at(point)
                if count >= 20:   # skip the first 20 points
                    #
                    # scale and draw the point
                    #
                    sx,sy = self.scale_point(point)
                    if self.verbose > 0:
                        print('n={}, point={}\t sx,sy = {},{}'.format(n, point, sx, sy))
                    point_count = self.record_point(point, (sx, sy))
                    if self.verbose > 0 and point_count > 1:
                        print(f"{sx},{sy}   {point_count} ")
                    if self.draw:
                        if self.verbose > 1:
                            print(f"{n}  draw: {sx},{sy} ")
                        self.pen.goto(sx, sy)
                        self.pen.pendown()
                        self.pen.dot(3)
                        self.pen.penup()
                count = count + 1
        if self.draw:
            print("Done")
            ttl.exitonclick()
            ttl.done()

In [6]:
#
# Barnsley Fern
#
bf1 = LinearFunction((0.0, 0.0, 0.0), (0.16, 0, 0))
bf2 = LinearFunction((0.85, 0.04, 0.0), (-0.04, 0.85, 1.6))
bf3 = LinearFunction((0.20, -0.26, 0.0), (0.23, 0.22, 1.60))
bf4 = LinearFunction((-0.15, -0.28, 0.0), (0.26, 0.24, 0.44))
print(str(bf2))
bf_ifs = IFS(bf1, 0.01)
bf_ifs.add_function(bf2, 0.85)
bf_ifs.add_function(bf3, 0.07)
bf_ifs.add_function(bf4, 0.07)
bf_ifs.ifs_df

f=((0.85, 0.04, 0.0)), ((-0.04, 0.85, 1.6))


,a,b,c,d,e,f,p,linear_function,name
0,0.00,0.00,0.0,0.16,0.00,0.00,0.01,"f0=((0.0, 0.0, 0.0)), ((0.16, 0, 0))",f0
1,0.85,0.04,0.0,-0.04,0.85,1.60,0.85,"f1=((0.85, 0.04, 0.0)), ((-0.04, 0.85, 1.6))",f1
2,0.20,-0.26,0.0,0.23,0.22,1.60,0.07,"f2=((0.2, -0.26, 0.0)), ((0.23, 0.22, 1.6))",f2
3,-0.15,-0.28,0.0,0.26,0.24,0.44,0.07,"f3=((-0.15, -0.28, 0.0)), ((0.26, 0.24, 0.44))",f3


In [7]:
#
# run the game to get scaling data
#
gridsize = (200, 200)
cg = ChaosGame(bf_ifs, draw=False, verbose=0, size=gridsize)
cg.run(limit = 500)
points_df = cg.points_df
hist_df = cg.histogram()
print(points_df.head())
print(cg.min_max())
print(cg.point_range())
print(cg.prob_sums)

               point       x       y       scaled_point  scaleX  scaleY count
0  (-1.5589, 1.7823) -1.5589  1.7823  (-1.5589, 1.7823) -1.5589  1.7823     1
1  (-1.2538, 3.1773) -1.2538  3.1773  (-1.2538, 3.1773) -1.2538  3.1773     1
2  (-1.0769, 2.0106) -1.0769  2.0106  (-1.0769, 2.0106) -1.0769  2.0106     1
3  (-0.4014, 0.6425) -0.4014  0.6425  (-0.4014, 0.6425) -0.4014  0.6425     1
4  (-0.3155, 2.1622) -0.3155  2.1622  (-0.3155, 2.1622) -0.3155  2.1622     1
{'minX': -3.1011, 'minY': -0.1229, 'maxX': 2.2603, 'maxY': 9.8646}
(5.3614, 9.9875)
0    0.01
1    0.86
2    0.93
3    1.00
Name: p, dtype: float64


In [15]:
point_stats = cg.min_max()
scaler = PointScaler(point_stats, gridsize=gridsize)
cg = ChaosGame(bf_ifs, draw=False, verbose=0, point_scaler=scaler, reps=1)
cg.run(limit = 1000)

In [16]:
# hist_df.query('count > 2')
hist_df = cg.histogram()
hist_df.groupby(by='count',axis=0).count()

,scaleX,scaleY
count,,
1,722,722
2,106,106
3,21,21
4,1,1


In [17]:
cg.draw_points()

Done


Terminator: 

In [35]:
# show convergence
point = (random.uniform(-1,1),random.uniform(-1,1))
linear_function = f2
for i in range(20):
    # print(point)
    point = linear_function.evaluate_at(point)

In [5]:
#
# Sierpinski
#
s1 = LinearFunction((0.5, 0.0, 0.0), (0, 0.5, 0.5))
s2 = LinearFunction((.5, 0, .5), (0, .5, .5))
s3 = LinearFunction((.5, 0, .25), (0, .5, 0))

sierpinski = IFS(s1, 0.3333)
sierpinski.add_function(s2, 0.3333)
sierpinski.add_function(s3, 0.3333)
sierpinski.ifs_df

,a,b,c,d,e,f,p,linear_function,name
0,0.5,0.0,0.00,0.0,0.5,0.5,0.3333,"f0=((0.5, 0.0, 0.0)), ((0, 0.5, 0.5))",f0
1,0.5,0.0,0.50,0.0,0.5,0.5,0.3333,"f1=((0.5, 0, 0.5)), ((0, 0.5, 0.5))",f1
2,0.5,0.0,0.25,0.0,0.5,0.0,0.3333,"f2=((0.5, 0, 0.25)), ((0, 0.5, 0))",f2


In [6]:
# show convergence
point = (random.uniform(-1,1),random.uniform(-1,1))
linear_function = s3
for i in range(20):
    #print(point)
    point = linear_function.evaluate_at(point)

In [7]:
#
# get the scaling information
#
cg2 = ChaosGame(sierpinski, draw=False, verbose = 0)
cg2.run(limit = 1000)
print(cg2.points_df.head())
print(cg2.min_max())
print(cg2.point_range())

              point       x       y scaled_point scaleX scaleY count
0  (0.8108, 0.9997)  0.8108  0.9997   (53, -215)     53   -215     1
1  (0.6554, 0.4999)  0.6554  0.4999   (43, -234)     43   -234     1
2    (0.5777, 0.25)  0.5777  0.2500   (38, -243)     38   -243     1
3   (0.2888, 0.625)  0.2888  0.6250   (19, -229)     19   -229     1
4  (0.3944, 0.3125)  0.3944  0.3125   (26, -240)     26   -240     1
{'minX': 0.0089, 'minY': 0.0149, 'maxX': 0.9903, 'maxY': 1.0}
(0.9814, 0.9851)


In [13]:
point_stats = cg2.min_max()
scaler = PointScaler(point_stats, gridsize=(400,400))
cg2 = ChaosGame(sierpinski, draw=True, verbose=0, point_scaler=scaler)
cg2.run(limit = 10000)

Done


Terminator: 

In [8]:
cg2.points_df

,point,x,y,scaled_point,scaleX,scaleY,count
0,"(0.8108, 0.9997)",0.8108,0.9997,"(53, -215)",53,-215,1
1,"(0.6554, 0.4999)",0.6554,0.4999,"(43, -234)",43,-234,1
2,"(0.5777, 0.25)",0.5777,0.2500,"(38, -243)",38,-243,1
3,"(0.2888, 0.625)",0.2888,0.6250,"(19, -229)",19,-229,1
4,"(0.3944, 0.3125)",0.3944,0.3125,"(26, -240)",26,-240,1
...,...,...,...,...,...,...,...
975,"(0.5687, 0.9979)",0.5687,0.9979,"(37, -215)",37,-215,1
976,"(0.2843, 0.999)",0.2843,0.9990,"(18, -215)",18,-215,1
977,"(0.3921, 0.4995)",0.3921,0.4995,"(25, -234)",25,-234,1
978,"(0.4461, 0.2497)",0.4461,0.2497,"(29, -243)",29,-243,1


In [14]:
#
# update the count of an existing point
#
scaled_point = (43, -234)
df = cg2.points_df
bs = df['scaled_point'].isin([scaled_point])
ind = bs.idxmax()
print(ind)
if bs.iloc[ind]:
    count = df.iloc[ind]['count']
    print(f"count = {count}")
    cg2.points_df.loc[(ind, 'count')]= count + 1
print(cg2.points_df)

1
count = 1
                point       x       y scaled_point scaleX scaleY count
0    (0.8108, 0.9997)  0.8108  0.9997   (53, -215)     53   -215     2
1    (0.6554, 0.4999)  0.6554  0.4999   (43, -234)     43   -234     2
2      (0.5777, 0.25)  0.5777  0.2500   (38, -243)     38   -243     1
3     (0.2888, 0.625)  0.2888  0.6250   (19, -229)     19   -229     1
4    (0.3944, 0.3125)  0.3944  0.3125   (26, -240)     26   -240     1
..                ...     ...     ...          ...    ...    ...   ...
975  (0.5687, 0.9979)  0.5687  0.9979   (37, -215)     37   -215     1
976   (0.2843, 0.999)  0.2843  0.9990   (18, -215)     18   -215     1
977  (0.3921, 0.4995)  0.3921  0.4995   (25, -234)     25   -234     1
978  (0.4461, 0.2497)  0.4461  0.2497   (29, -243)     29   -243     1
979   (0.473, 0.1249)  0.4730  0.1249   (31, -247)     31   -247     1

[980 rows x 7 columns]


In [13]:
bs

0      False
1      False
2      False
3      False
4      False
       ...  
975    False
976    False
977    False
978    False
979    False
Name: scaled_point, Length: 980, dtype: bool

In [41]:
gb = cg2.points_df[['scaled_point', 'scaleX', 'scaleY']].groupby('scaled_point').count()
gb[gb['scaleX'] > 1]

,scaleX,scaleY
scaled_point,,
"(0, -215)",2,2
"(1, -216)",4,4
"(2, -217)",3,3
"(3, -218)",2,2
"(3, -216)",5,5
...,...,...
"(62, -218)",4,4
"(63, -216)",3,3
"(64, -216)",2,2


In [42]:
df = pd.DataFrame(columns=['point','x','y'])
point = (random.uniform(-1,1),random.uniform(-1,1))
l = pd.Series(data={'point':point, 'x':point[0], 'y':point[1]})
df.append(l, ignore_index=True)

,point,x,y
0,"(0.686643069442989, -0.8763333390683676)",0.686643,-0.876333


In [8]:
#
# draw outline of region size 800 x 800. (0,0) is center of screen
# top right = (400, 400), bottom left = (-400, -400)
#
pos1 = ttl.position()
print(pos1)
ttl.color('red', 'green')
ttl.circle(10)
ttl.forward(400)
ttl.left(90)
ttl.forward(400)
ttl.left(90)
ttl.forward(400)
ttl.circle(10)
ttl.left(90)
ttl.forward(800)
ttl.dot(5, 'blue')
ttl.right(90)
ttl.forward(400)
ttl.dot(5, 'blue')
pos2 = ttl.position()
print(pos2)
ttl.exitonclick()
ttl.done()

(0.00,0.00)
(-400.00,-400.00)


Terminator: 